In [67]:
import numpy as np
import math

In [68]:
class MakeReceivedImg(object):
    def __init__(self, numberOfLEDs=16, gaussSigma=0.45, kernelSize=9, maxLum=1, minLum=0, boxNoise=0.3):
        """
        Initial functions of this class
        """
        self.numberOfLEDs = numberOfLEDs
        self.sqrtnumberofled = math.sqrt(numberOfLEDs)
        self.gaussSigma = gaussSigma
        self.kernelSize = kernelSize
        self.maxLum = maxLum
        self.minLum = minLum
        self.boxNoise = boxNoise
    
    def RandomLEDs(self):
        """
        Making randomly blinking LED and store them in array(numpy) 
        numberOfLEDs: Number of LEDs. Default = 16
        max_lum_value: maximum luminance value of LEDs
        min_lum_value: minimum luminance value of LEDs
        """
        # Generating random integers [0,1]
        blinking_leds = np.random.randint(0, 2, self.numberOfLEDs)

        # aplly max/min luminance value depending on its blinking condition
        for index, value in enumerate(blinking_leds):
          if value == 0:
            blinking_leds[index] = self.minLum
          else:
            blinking_leds[index] = self.maxLum

        # [sqrt(len(blinking_leds)]
        led_len = math.sqrt(len(blinking_leds))
        assert led_len % 2 == 0, 'Expected: 0 == led_len%2 \n Actual: 1 == led_len%2'

        # change dtype float to int
        led_len = int(led_len)

        # reshape led_len to two-d array from one-d array
        led_condition = np.reshape(blinking_leds, (led_len, led_len))

        return np.array(blinking_leds)


    def GaussChannelAndInv(self):
        """
        Creating channel parameters based on Gauss function
        """

        y_scale = int(self.sqrtnumberofled)
        x_scale = int(self.sqrtnumberofled)        

        # 配列の確保
        gauss_channel = [[0 for i in range(y_scale * x_scale)] for j in range(y_scale * x_scale)]

        # 自身と他の画素から影響を受ける画素の決定
        for y in range(0, y_scale):
            for x in range(0, x_scale):
                target_pixel = y * x_scale + x  

                # target_pixelへ影響を与えるinfluence_pixelを決定
                for i in range(y_scale):
                    for j in range(x_scale):

                        influence_pixel = i * x_scale + j

                        # ガウシアンフィルの作成
                        distance = (j - x) * (j - x) + (i - y) * (i - y)
                        weight = math.exp(-distance / (2.0 * self.gaussSigma * self.gaussSigma))  * (1 / (2.0 * math.pi * self.gaussSigma * self.gaussSigma))

                        if ((j - x) * (j - x)) >= self.kernelSize or ((i - y) * (i - y)) >= self.kernelSize:
                            weight = 0
                        else:
                            gauss_channel[target_pixel][influence_pixel] = weight

                gauss_channel = np.array(gauss_channel)

        # 逆行列の作成
        inv_channel = gauss_channel.copy()
        inv_gauss_channel = np.linalg.inv(inv_channel)
        return gauss_channel, inv_gauss_channel
      
      
    def Filtering(self, leds):
        """
        Aplly filtering to LED arrays created in RandomLEDs
        """

        channel, invchannel = self.GaussChannelAndInv()
        #pixel_values = np.zeros(self.numberOfLEDs)

        pixel_values = np.dot(channel, leds)

       # pixel_values = np.dot(invchannel, pixel_values)

        return pixel_values
 
        """  
        def InvGaussChannel(self):

        #逆行列を施す


        channel = self.GaussChannel()
        inv_gauss_channel = np.linalg.inv(channel)
        return inv_gauss_channel
            """      
  
  
    def GetNoise(self):
        """
        Generate noise by boxmuller method
        """

        from scipy.stats import uniform

        # 独立した一様分布からそれぞれ一様乱数を生成
        np.random.seed()
        N = self.numberOfLEDs
        rv1 = uniform(loc=0.0, scale=1.0)
        rv2 = uniform(loc=0.0, scale=1.0)
        U1 = rv1.rvs(N)
        U2 = rv2.rvs(N)

        # Box-Mullerアルゴリズムで正規分布に従う乱数に変換
        # 2つの一様分布から2つの標準正規分布が得られる
        X1 = np.sqrt(-2*np.log(U1)) * np.cos(2*np.pi*U2) * self.boxNoise
        #X2 = np.sqrt(-2 * np.log(U1)) * np.sin(2 * np.pi * U2) * boxNoise

        return X1
    
    def ReceivedImg(self):
        """
        Create received image
        """
        leds = self.RandomLEDs()
        pix = self.Filtering(leds)
        noise = self.GetNoise()
        receivedimg = pix + noise
        return receivedimg, leds
    
    def create_dataset(self, loop=100):
        """
        Create dataset
        """
        
        # LED condition (1/0)
        led_condition = np.empty((0))
        pixel_value = np.empty((0))
        
        for iteration in range(loop):
            receivedimg, led_condi = self.ReceivedImg()
            pixel_value = np.hstack((pixel_value, receivedimg))
            led_condition = np.hstack((led_condition, led_condi))
            
        led_condition_2d = np.reshape(led_condition, (loop, self.numberOfLEDs))
        pixel_value_2d = np.reshape(pixel_value, (loop, self.numberOfLEDs))
        
        return led_condition_2d, pixel_value_2d

if __name__ == '__main__':
    mri = MakeReceivedImg()

In [344]:
mri = MakeReceivedImg(boxNoise=0.1)
led, pix = mri.create_dataset(loop=100)
print(pix.shape)
print(led.shape)

(100, 16)
(100, 16)


In [345]:
def estimate_channel(led_pattern, pixel_value):
    """
    提案手法でのチャネル推定
    """
    
    #led, pix = mri.create_dataset(loop=10)
    inv_led = np.linalg.pinv(led_pattern)
    len_loop, len_led = led_pattern.shape
    
    estimated_channel = np.array([[0 for i in range(len_led)] for j in range(len_led)], dtype='float32')
    
    for idx_led in range(len_led):
        # indx_led番目のLEDの点滅状態を格納
        tmp_led = []

        for idx_loop in range(len_loop):
            tmp_led.append(pixel_value[idx_loop][idx_led])
        
        # idx_led番目のLEDのチャネルを格納
        tmp_channel = np.dot(inv_led, tmp_led)
        for i in range(len_led):
            estimated_channel[idx_led][i] = tmp_channel[i]
            
    return np.array(estimated_channel, dtype='float32')

In [346]:
est_ch = estimate_channel(led, pix)

In [347]:
def estimate_channel_conv(numleds=16, numimages=100):
    """
    個別点灯画像の1000枚の平均を用いたチャネル推定
    """
    estimated_channel = np.array([[0 for i in range(numleds)] for j in range(numleds)], dtype='float32')
    for led in range(numleds):
        leds = np.zeros(numleds, dtype='int32')
        leds[led] = 1
        total_pix_val = [0]
        for image in range(numimages):
            pix = mri.Filtering(leds)
            noise = mri.GetNoise()
            pix_val = pix + noise
            total_pix_val += pix_val / numimages
        
        for i in range(numleds):
            estimated_channel[i][led] = total_pix_val[i]
    
    return estimated_channel

In [348]:
ch_pro = estimate_channel(led, pix)
ch_conv = estimate_channel_conv()

In [349]:
print(ch_conv)

[[ 7.93806493e-01  6.75217733e-02 -1.18247215e-02  6.71183132e-03
   7.41583779e-02 -9.51517187e-03 -3.10061476e-03  1.39041403e-02
   2.69326102e-03  3.33061479e-02 -9.37138870e-03 -6.30186033e-03
  -4.18761512e-03 -6.37299707e-03 -8.35717097e-03  1.46186096e-03]
 [ 6.41625449e-02  7.74386168e-01  8.39855522e-02 -1.79530624e-02
  -2.11132737e-03  7.39835873e-02  5.13093639e-03  3.05520254e-03
   2.03433018e-02 -2.80893338e-03  2.73003485e-02  8.62715766e-03
  -3.84312659e-03  4.41803085e-03 -1.94774417e-03  4.32870071e-03]
 [ 1.81556516e-03  6.46018684e-02  7.84536481e-01  8.35964531e-02
  -7.41697615e-03 -2.02143565e-03  5.51083684e-02  1.13956770e-03
  -3.71027342e-03  1.12096965e-02 -1.06916588e-03  1.12853320e-02
   8.62696301e-03 -1.74589530e-02 -7.42004113e-03  8.34289267e-07]
 [ 9.30374837e-04 -5.90504613e-03  6.50192797e-02  7.91224420e-01
   6.77753380e-03  6.85865292e-03  6.73940312e-03  6.19839132e-02
  -8.63621477e-03 -1.40463589e-02  1.17011694e-02 -1.32427616e-02
   5.69

In [350]:
pixel_values = pix
answer_signals = led

In [351]:
def modulation_zf(pixelValue, answers, gauss, loop_times):
    """
    推定したチャネルからZFにより復調を行う
    """
    inv_gauss = np.linalg.pinv(gauss)
    error = 0
    _, len_led = pixelValue.shape
    
    for loop in range(loop_times):
        
        pixel_value = pixelValue[loop][:]
        answer = answers[loop][:]
    
        estimated_signals = np.dot(inv_gauss, pixel_value)
        estimated_signals_copy = estimated_signals.copy()
        estimated_signals_copy = np.where(estimated_signals_copy > 0.5, 1, 0)
        
        error += np.sum(estimated_signals_copy != answer)
        
    return error / (loop_times*len_led)

In [352]:
ber_pro = modulation_zf(pixel_values, answer_signals, ch_pro, 100)
ber_conv = modulation_zf(pixel_values, answer_signals, ch_conv, 100)

In [353]:
print(ber)
print(ber_conv)

0.14875
0.0
